# 5. Analisi e Visualizzazione dei Cluster

**Obiettivo:** Interpretare i cluster per definire i diversi "stili di gioco". Questa è la fase finale, dove trasformo i numeri in profili comprensibili.

1.  **Caricamento Dati:** Leggo i risultati del clustering.
2.  **Calcolo dei Profili Medi:** Calcolo le statistiche medie per ogni cluster per creare un "identikit" statistico.
3.  **Visualizzazione Comparativa (Radar Chart):** Creo un radar chart per confrontare visivamente i profili.
4.  **Identificazione di Giocatori Rappresentativi:** Trovo giocatori noti per ogni cluster per dare un volto ai profili.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pyspark.sql.functions import col
from sklearn.preprocessing import minmax_scale

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.clustering.models import get_cluster_profiles
from src.config.spark_config import SPARK_CONFIG
from src.utils.helpers import get_spark_session

project_root = os.path.abspath(os.path.join('..'))
processed_data_dir = os.path.join(project_root, "data", "processed")
reports_dir = os.path.join(project_root, "reports", "figures")
clustered_path = os.path.join(processed_data_dir, "players_clustered.parquet")
adv_metrics_path = os.path.join(processed_data_dir, "players_advanced_metrics.parquet")

spark = get_spark_session(
    app_name="NBA_Cluster_Analysis",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

sns.set_style("whitegrid")

### Fase 1: Caricamento dei Risultati e Preparazione Finale

Carico i risultati del clustering, rinumero i cluster da 1 a 6 (invece di 0-5) per renderli più leggibili e li unisco con le statistiche complete.

In [ ]:
df_clustered_raw = spark.read.parquet(clustered_path)
df_full_stats = spark.read.parquet(adv_metrics_path)

if 'mp_per_game' not in df_full_stats.columns:
    from src.data_processing.normalization import add_per_game_metrics
    df_full_stats = add_per_game_metrics(df_full_stats)

df_clustered_renum = df_clustered_raw.withColumn("cluster_id", col("cluster_id") + 1)
print("Numerazione dei cluster aggiornata a 1-6.")

df_final_analysis = df_full_stats.join(
    df_clustered_renum.select("player", "season", "cluster_id"),
    ["player", "season"],
    "inner"
)
print("Dati di clustering uniti con le statistiche complete.")

feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'
]

### Fase 2: Analisi dei Profili Medi dei Cluster

Per capire cosa definisce ogni cluster, calcolo le statistiche medie delle feature per tutti i giocatori di quel gruppo.

In [ ]:
cluster_profiles_df = get_cluster_profiles(df_clustered_renum, feature_cols)
cluster_profiles_pd = cluster_profiles_df.toPandas()

print("Profili Statistici Medi per Cluster (1-6):")
display(cluster_profiles_pd.set_index('cluster_id'))

#### Interpretazione della tabella

Dalla tabella, inizio già a vedere delle tendenze: un cluster forte in punti e assist, un altro in rimbalzi e stoppate, un altro con pochissime palle perse. Queste sono le prime tracce per definire i profili.

### Fase 3: Visualizzazione dei Profili con Radar Chart

Il radar chart è perfetto per confrontare più variabili tra diverse categorie. Mi darà una visione d'insieme potente e immediata degli stili di gioco.

In [ ]:
feature_cols_reordered = [
    'pts_per_36_min',
    'ts_pct_calc',
    'ast_per_36_min',
    'tov_per_36_min',
    'stl_per_36_min',
    'blk_per_36_min',
    'trb_per_36_min'
]

labels = [c.replace('_per_36_min', '').replace('_calc', '').upper() for c in feature_cols_reordered]

num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

profiles_scaled = cluster_profiles_pd.copy()
for col_name in cluster_profiles_pd.columns:
    if col_name.startswith('avg_'):
        profiles_scaled[col_name] = minmax_scale(profiles_scaled[col_name])

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

for i, row in profiles_scaled.iterrows():
    values = row[[f'avg_{col}' for col in feature_cols_reordered]].tolist()
    values += values[:1]
    ax.plot(angles, values, label=f"Cluster {row['cluster_id']}", linewidth=2)
    ax.fill(angles, values, alpha=0.15)

ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, size=12)
plt.title('Profili Comparati dei Cluster di Giocatori NBA', size=20, color='navy', y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)
plt.savefig(os.path.join(reports_dir, 'cluster_radar_chart.png'))
plt.show()

#### Interpretazione del grafico

Il radar chart è spettacolare. Ogni poligono è uno stile di gioco. Vedo chiaramente un poligono (le "All-Around Stars") che si estende molto su punti e assist, un altro con picchi su rimbalzi e stoppate e uno molto piccolo al centro per i giocatori a basso utilizzo. Le differenze sono evidenti.

### Fase 4: Esempi di Giocatori per Cluster

Per dare un volto ai numeri, identifico alcuni giocatori rappresentativi per ogni cluster, ordinandoli per la statistica più rilevante per quel profilo.

In [ ]:
cluster_profiles_map = {
    1: "Ali Forti Moderne / Marcatori-Rimbalzisti",
    2: "Playmaker Puri / Organizzatori di Gioco",
    3: "Giocatori di Ruolo a Basso Utilizzo",
    4: "All-Around Stars / Motori Offensivi",
    5: "Giocatori Affidabili a Controllo Rischio",
    6: "Ancore Difensive / Specialisti del Canestro"
}

sort_logic = {
    1: {"column": "pts_per_36_min", "ascending": False},
    2: {"column": "ast_per_36_min", "ascending": False},
    3: {"column": "mp_per_game", "ascending": True},
    4: {"column": "pts_per_36_min", "ascending": False},
    5: {"column": "tov_per_36_min", "ascending": True},
    6: {"column": "blk_per_36_min", "ascending": False}
}

print("Giocatori Rappresentativi per Cluster:")
for i, logic in sort_logic.items():
    profile = cluster_profiles_map.get(i, f"Profilo Sconosciuto {i}")
    order_col = logic["column"]
    is_ascending = logic["ascending"]

    print(f"\n--- CLUSTER {i}: {profile} (Ordinato per: {order_col} {'ASC' if is_ascending else 'DESC'}) ---")
    (df_final_analysis.filter(col('cluster_id') == i)
                     .orderBy(col(order_col).asc() if is_ascending else col(order_col).desc())
                     .select('player', 'season', 'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 'tov_per_36_min', 'ts_pct_calc', 'mp_per_game')
                     .show(5, truncate=False)
    )

#### Interpretazione dell'output

Le tabelle mi mostrano nomi che corrispondono perfettamente ai profili che ho definito: Luka Dončić tra le All-Around Stars, Rudy Gobert tra le Ancore Difensive, Chris Paul tra i Playmaker Puri. Questa è la validazione finale più importante del mio lavoro.

### Conclusione e Prossimi Passi

Ho dato un senso ai cluster, trasformandoli in profili di giocatori ben definiti, etichettati e validati.

Nel prossimo e ultimo notebook, `06_presentation_visuals.ipynb`, userò queste intuizioni per creare i grafici per la presentazione.

In [ ]:
spark.stop()